In [5]:
from benchmark import *
%matplotlib inline

In [6]:
helpers = 'pdb'
sp = 'sp'
xp = 'xp'

def results(mcss_func, name):
    data = pd.concat([get_data(helpers, sp, False), get_data(helpers, xp, False)])

    data = data[mcss_func(data.mcss)]

    ligand = data.xs((helpers, 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                      level=('helpers', 'mode', 'n_ligs', 'features', 'alpha')).filter(regex='rmsd|correct')
    ligand = filter_to_ubiquitous_ligands(ligand)
    target = ligand.groupby(level=list(range(len(ligand.index.levels)-1))).mean().dropna()
    
    family = ligand.groupby(level=list(range(len(ligand.index.levels)-2))).mean()

    target = pd.merge(target.loc[sp], target.loc[xp], left_index=True, right_index=True)
    family = pd.merge(family.loc[sp], family.loc[xp], left_index=True, right_index=True)

    family['SP Top'] = family['glide_correct_x']
    family['SP ComBind'] = family['combind_correct_x']
    family['SP Best'] = family['best_correct_x']
    family['XP Top'] = family['glide_correct_y']
    family['XP ComBind'] = family['combind_correct_y']
    family['XP Best'] = family['best_correct_y']
    family = family.filter(regex='P')

    family['version'] = name
    family.reset_index(inplace=True)
    family.set_index(['version', 'family'], inplace = True)
    return drug_average(family)

In [7]:
pd.concat([results(lambda x: x > -1, 'All'),
           results(lambda x: x >= 0.5, 'Shared scaffold'),
           results(lambda x: x < 0.5, 'No shared scaffold')])

,SP Top,SP ComBind,SP Best,XP Top,XP ComBind,XP Best
version,,,,,,
All,0.486355,0.662026,0.839222,0.525540,0.576594,0.688009
Shared scaffold,0.572885,0.774982,0.852612,0.697514,0.753773,0.812193
No shared scaffold,0.443077,0.604299,0.812536,0.452884,0.499726,0.629964


# Number of ligands for each target

In [12]:
data = get_data('pdb', 'sp', False)
data = add_correct(data, thresh = 2.0)

all_ligands = data.xs(('sp', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                  level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

diverse = data[data.mcss < 0.5].xs(('sp', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                  level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

diverse_correct = data[(data.mcss < 0.5) & data.best_correct].xs(
                    ('sp', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                     level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

df = pd.DataFrame()
df['all'] = all_ligands.groupby(level='protein').count().iloc[:, 0]
df['diverse'] = diverse.groupby(level='protein').count().iloc[:, 0]
df['diverse_correct'] = diverse_correct.groupby(level='protein').count().iloc[:, 0]
df

,all,diverse,diverse_correct
protein,,,
5HT2B,5,5,5
AR,19,12,10
B1AR,11,6,6
B2AR,7,4,4
BACE1,20,19,7
BRD4,16,13,7
CDK2,20,20,17
DAT,8,8,7
DHFR,20,20,15


In [13]:
df.sum()

all                421
diverse            327
diverse_correct    248
dtype: int64